In [1]:
# from google.colab import drive
# drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
# !pip install praw
# !pip install sentencepiece
# !pip install datasets
# !pip install accelerate -U
# !pip install tweepy
# !pip install python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.0/191.0 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 2.7 MB/s eta 0:00:00


In [1]:

import os
from dotenv import load_dotenv, find_dotenv
import pandas as pd
import sys
sys.path.insert(0,'../config')
from config_predict import *
import tweepy
from judgesComments import *
import praw
import sentencepiece
import torch
from torch.utils.data import DataLoader, TensorDataset
# from datasets import load_metric
from transformers import BigBirdModel
from transformers import (BigBirdForSequenceClassification, BigBirdTokenizer
                          , Trainer, TrainingArguments)
import numpy as np
from datetime import datetime
from random import choice


/home/walkerped/anaconda3/envs/aita/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# read in current pred sheet
if os.path.exists(current_sheet_path):
  current_sheet_df = pd.read_csv(current_sheet_path, index_col = 0)
else:
  current_sheet_df = None

# load up the entries as environment variables
load_dotenv(dotenv_path)

client_id = os.environ.get("CLIENT_ID")
client_secret = os.environ.get("CLIENT_SECRET")
user_agent = os.environ.get("USER_AGENT")

print(current_sheet_path)
print(current_sheet_df)


../data/app_tracking/prediction_sheets/unresolved_predictions.csv
                                                title    logits  pred  \
0   AITA - Boyfriends Dog Has Tumors and Flea Infe... -0.010123     0   
1   AITA for leaving my grandparents house after m... -0.013245     0   
2   AITAH for saying my BF’s sister’s house isn’t ...  1.061646     1   
3   WIBTA? Harassed by ex boyfriends friends and f... -0.009613     0   
4   AITA for not communicating well enough for a t...  1.062508     1   
5         AITA for telling my partener to lighten up?  1.061629     1   
6   AITA For not wanting to go to school with a fr... -0.004504     0   
7   AITA for making a sad face when my boyfriend r... -0.009119     0   
8   AITA for telling my father in law i won’t cook... -0.011271     0   
9   AITA for telling my kid he is welcome to move ...  1.068286     1   
10  AITA for forcing my daughter to invite her ste...  1.059065     1   
11  WIBTA for buying a GPU plushie, putting it in ...  1.0

In [3]:
# Authenticate
reddit = praw.Reddit(
    client_id=client_id,
    client_secret=client_secret,
    user_agent=user_agent
)

# Get the subreddit
subreddit = reddit.subreddit('AmITheAsshole')

# Fetch new posts
new_posts = subreddit.new(limit=n_posts)

raw_post_df = pd.DataFrame(
    [ vars(post) for post in subreddit.new(limit=n_posts)]
    )

vars_to_keep = ['title','selftext','link_flair_text','created_utc','url']

index_no_flair = raw_post_df[raw_post_df['link_flair_text'].notnull()].index

abbrev_post_df = raw_post_df[vars_to_keep].drop(index_no_flair)

if current_sheet_df is not None:
  filtered_post_df = abbrev_post_df[
      ~abbrev_post_df['title'].isin(current_sheet_df['title'])
      ]
else:
  filtered_post_df = current_sheet_df

filtered_post_df['post_date'] = filtered_post_df['created_utc'].apply(
    lambda x: datetime.utcfromtimestamp(x).strftime('%Y-%m-%d')
    )

filtered_post_df['titles_and_texts'] = (filtered_post_df['title']
                                          + filtered_post_df['selftext'])

filtered_post_df['date_created'] = datetime.today().strftime('%m-%d-%Y')

post_df = filtered_post_df.drop(['created_utc','selftext'], axis=1)

display(post_df.head())


/tmp/ipykernel_36362/2830038395.py:32: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  lambda x: datetime.utcfromtimestamp(x).strftime('%Y-%m-%d')
/tmp/ipykernel_36362/2830038395.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_post_df['post_date'] = filtered_post_df['created_utc'].apply(
/tmp/ipykernel_36362/2830038395.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

,title,link_flair_text,url,post_date,titles_and_texts,date_created
0,AITA for asking my husband to get a separate b...,None,https://www.reddit.com/r/AmItheAsshole/comment...,2024-03-22,AITA for asking my husband to get a separate b...,03-22-2024
1,WIBTA if I refused to give my parents 25% of m...,None,https://www.reddit.com/r/AmItheAsshole/comment...,2024-03-22,WIBTA if I refused to give my parents 25% of m...,03-22-2024
2,AITA for not wanting to give my mother my money?,None,https://www.reddit.com/r/AmItheAsshole/comment...,2024-03-22,AITA for not wanting to give my mother my mone...,03-22-2024
3,AITA for confronting my 'friend' about plannin...,None,https://www.reddit.com/r/AmItheAsshole/comment...,2024-03-22,AITA for confronting my 'friend' about plannin...,03-22-2024
4,AITA for refusing to help my sister after she ...,None,https://www.reddit.com/r/AmItheAsshole/comment...,2024-03-22,AITA for refusing to help my sister after she ...,03-22-2024


In [4]:
def assign_torch_device(quiet=False):
  if torch.cuda.is_available():
      device = torch.device("cuda")  # GPU
      if not quiet:
        print("GPU available. Using GPU.")
  else:
      device = torch.device("cpu")  # CPU
      if not quiet:
        print("GPU is not available. Using CPU.")
  return device

device = assign_torch_device()

def aita_prep_data(post_list):
  tokenizer = BigBirdTokenizer.from_pretrained('google/bigbird-roberta-base')

  post_tokens = tokenizer(post_list, padding=True
                          , truncation=True, return_tensors="pt")

  return TensorDataset(post_tokens['input_ids']
                                    , post_tokens['attention_mask'])

to_predict_dataset = aita_prep_data(post_df['titles_and_texts'].tolist())

def aita_predict(model_path, data_to_predict):

  def data_collector(features):
    batch = {}
    batch['input_ids'] = torch.stack([f[0] for f in features])
    batch['attention_mask'] = torch.stack([f[1] for f in features])
    return batch

  model_loaded = BigBirdForSequenceClassification.from_pretrained(
    model_path).to(device)

  training_args = TrainingArguments("test_trainer",report_to='none')

  # Initialize Trainer
  trainer = Trainer(
      model=model_loaded,
      args=training_args,
      data_collator=data_collector,
  )

  return trainer.predict(data_to_predict)
pred = aita_predict(model_path,to_predict_dataset)


GPU available. Using GPU.


/home/walkerped/anaconda3/envs/aita/lib/python3.12/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
Input ids are automatically padded from 951 to 960 to be a multiple of `config.block_size`: 64
100%|██████████| 17/17 [00:06<00:00,  2.50it/s]


In [5]:
logits_list = pred.predictions.reshape(-1)
bin_pred_list = (pred.predictions > 0.5).astype(np.int32).reshape(-1)

pred_df = post_df[['title', 'post_date', 'url', 'date_created']]

pred_df['logits'] = logits_list

pred_df['pred'] = bin_pred_list

display(pred_df.head())

/tmp/ipykernel_36362/1543316981.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pred_df['logits'] = logits_list


,title,post_date,url,date_created,logits,pred
0,AITA for asking my husband to get a separate b...,2024-03-22,https://www.reddit.com/r/AmItheAsshole/comment...,03-22-2024,0.929160,1
1,WIBTA if I refused to give my parents 25% of m...,2024-03-22,https://www.reddit.com/r/AmItheAsshole/comment...,03-22-2024,0.017975,0
2,AITA for not wanting to give my mother my money?,2024-03-22,https://www.reddit.com/r/AmItheAsshole/comment...,03-22-2024,0.002832,0
3,AITA for confronting my 'friend' about plannin...,2024-03-22,https://www.reddit.com/r/AmItheAsshole/comment...,03-22-2024,1.058804,1
4,AITA for refusing to help my sister after she ...,2024-03-22,https://www.reddit.com/r/AmItheAsshole/comment...,03-22-2024,1.003941,1


In [6]:
# get 10% most extreme pos and neg predictions, respectively
percentile_thresh=.1

def subset_extremes(in_df, case_pred_col_name, thresholding_col_name
                    , percentile_thresh=.1, quiet=False):
  threshs = []
  for case_type in [0,1]:

    if case_type == 0:
      perc_thresh = percentile_thresh
    elif case_type == 1:
      perc_thresh = 1-percentile_thresh

    threshs.append(in_df[(in_df[case_pred_col_name] == case_type)][
        thresholding_col_name].quantile(q=perc_thresh))


  df_subset = in_df.loc[
      (in_df[thresholding_col_name] < threshs[0])
        | (in_df[thresholding_col_name] > threshs[1])
      ]
  if not quiet:
      print(
          f'{thresholding_col_name} threshold for percentile of '
          f'{percentile_thresh} is {threshs[0]:.3} where {case_pred_col_name} '
          f'= 0 and {threshs[1]:.3} where {case_pred_col_name} = 1'
          )

  return df_subset

pred_percentile_df = subset_extremes(pred_df, 'pred', 'logits')

display(pred_percentile_df.sort_values('logits'))


logits threshold for percentile of 0.1 is -0.00203 where pred = 0 and 1.06 where pred = 1


,title,post_date,url,date_created,logits,pred
108,AITA for telling my mother I don’t love her an...,2024-03-22,https://www.reddit.com/r/AmItheAsshole/comment...,03-22-2024,-0.013843,0
26,AITA for saying no to my entitled little sister.,2024-03-22,https://www.reddit.com/r/AmItheAsshole/comment...,03-22-2024,-0.012162,0
110,AITA For talking about wanting to get a tattoo...,2024-03-22,https://www.reddit.com/r/AmItheAsshole/comment...,03-22-2024,-0.012084,0
37,AITA for not finically supporting my family an...,2024-03-22,https://www.reddit.com/r/AmItheAsshole/comment...,03-22-2024,-0.009358,0
38,AITA for not texting my ex back about my perso...,2024-03-22,https://www.reddit.com/r/AmItheAsshole/comment...,03-22-2024,-0.009169,0
65,AITA for not wanting to switch schools for my ...,2024-03-22,https://www.reddit.com/r/AmItheAsshole/comment...,03-22-2024,-0.004778,0
87,WIBTA for wanting to end the friendship over h...,2024-03-22,https://www.reddit.com/r/AmItheAsshole/comment...,03-22-2024,-0.003596,0
102,AITA for body-shaming my step-sister in front ...,2024-03-22,https://www.reddit.com/r/AmItheAsshole/comment...,03-22-2024,-0.002806,0
60,AITA for being ungrateful?,2024-03-22,https://www.reddit.com/r/AmItheAsshole/comment...,03-22-2024,1.060106,1
130,AITA for not wanting my sister to have my room...,2024-03-22,https://www.reddit.com/r/AmItheAsshole/comment...,03-22-2024,1.060923,1


In [7]:
selected_preds_df = pred_percentile_df.sample(n=n_tweets)
print(selected_preds_df)

                                                title   post_date  \
65  AITA for not wanting to switch schools for my ...  2024-03-22   
47  WIBTA(18M) for thinking some of my friends mig...  2024-03-22   

                                                  url date_created    logits  \
65  https://www.reddit.com/r/AmItheAsshole/comment...   03-22-2024 -0.004778   
47  https://www.reddit.com/r/AmItheAsshole/comment...   03-22-2024  1.061206   

    pred  
65     0  
47     1  


In [8]:
def tweepy_auth():
  import tweepy

  consumer_key = os.environ.get("CONSUMER_KEY")
  consumer_secret = os.environ.get("CONSUMER_SECRET")
  access_token = os.environ.get("ACCESS_TOKEN")
  access_token_secret = os.environ.get("ACCESS_TOKEN_SECRET")

  auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
  auth.set_access_token(access_token, access_token_secret)
  return tweepy.Client(consumer_key=consumer_key,
                    consumer_secret=consumer_secret,
                    access_token=access_token,
                    access_token_secret=access_token_secret)

def smart_truncate(content, length=100, suffix='...'):
    content_strip = content.strip()
    if len(content_strip) <= length:
        return content_strip
    else:
        return content_strip[:length].rsplit(' ', 1)[0]+suffix

def update_df(existing_df,new_def):
  import pandas as pd

  concat_df = pd.concat([existing_df,new_def])
  return concat_df.drop_duplicates(ignore_index=True)

client = tweepy_auth()

intro_string = 'Judge bot has heard the case of'
ruling_string = 'Judge bot rules:'

label_name_dict = {0:'NTA', 1:'YTA'}

# create and post tweet
for index, row in selected_preds_df.iterrows():

  title_trunc = f"\"{smart_truncate(row['title'])}\""

  if row['pred']==0:
    comment_string = choice(judges_comments['nta'])
  if row['pred']==1:
    comment_string = choice(judges_comments['yta'])

  full_tweet_string = (
      f"{intro_string} {title_trunc}. {ruling_string} "
      f"{label_name_dict[row['pred']]}. {comment_string}  {row['url']}"
  )

  # post to twitter
  client.create_tweet(text=full_tweet_string)


new_pred_df = update_df(current_sheet_df,selected_preds_df)

new_pred_df.reset_index(drop=True).to_csv(path_or_buf=current_sheet_path)

print(new_pred_df)



                                                title    logits  pred  \
0   AITA - Boyfriends Dog Has Tumors and Flea Infe... -0.010123     0   
1   AITA for leaving my grandparents house after m... -0.013245     0   
2   AITAH for saying my BF’s sister’s house isn’t ...  1.061646     1   
3   WIBTA? Harassed by ex boyfriends friends and f... -0.009613     0   
4   AITA for not communicating well enough for a t...  1.062508     1   
5         AITA for telling my partener to lighten up?  1.061629     1   
6   AITA For not wanting to go to school with a fr... -0.004504     0   
7   AITA for making a sad face when my boyfriend r... -0.009119     0   
8   AITA for telling my father in law i won’t cook... -0.011271     0   
9   AITA for telling my kid he is welcome to move ...  1.068286     1   
10  AITA for forcing my daughter to invite her ste...  1.059065     1   
11  WIBTA for buying a GPU plushie, putting it in ...  1.062298     1   
12  AITA for setting limits on my little sister's .